![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Applied_Generative_AI/Healthcare_NLP_Agents_with_LLMs.ipynb)

# **JohnSnowLabs Healthcare NLP Agents with LLMs (Certification Trainings, July 2024)**

# Coding an LLM Agent with John Snow Labs Library (Healthcare NLP)

In [1]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

Saving spark_nlp_for_healthcare_spark_ocr_9272_540.json to spark_nlp_for_healthcare_spark_ocr_9272_540.json


In [52]:
spark

In [2]:
license_keys['JSL_VERSION']


'5.4.0'

In [3]:
license_keys['PUBLIC_VERSION']


'5.4.0'

In [4]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.2/579.2 kB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.9/398.9 kB 536.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.8 MB/s eta 0:00:00


In [5]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 5.4.0
Spark NLP_JSL Version : 5.4.0


In [6]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

In [7]:
!pip install -qU transformers accelerate flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 58.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.3 MB/s eta 0:00:00


## Information Extraction with LLMs

In [54]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [55]:
messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [56]:
%%time

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


 To solve the equation 2x + 3 = 7, follow these steps:

1. Subtract 3 from both sides of the equation:
   2x + 3 - 3 = 7 - 3
   2x = 4

2. Divide both sides of the equation by 2:
   2x/2 = 4/2
   x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.
CPU times: user 8.47 s, sys: 25.1 ms, total: 8.5 s
Wall time: 8.58 s


In [11]:
%%time

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


 To solve the equation 2x + 3 = 7, follow these steps:

1. Subtract 3 from both sides of the equation:
   2x + 3 - 3 = 7 - 3
   2x = 4

2. Divide both sides of the equation by 2:
   2x/2 = 4/2
   x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.
CPU times: user 8.61 s, sys: 0 ns, total: 8.61 s
Wall time: 9.3 s


In [57]:
user_prompt = f"I have a patient 65 yrs old with a history of diabetes and now suffers from COPD. Suggest treatment options."

output = pipe(user_prompt, **generation_args)

print (output[0]['generated_text'])



# Answer
For a 65-year-old patient with a history of diabetes and newly diagnosed COPD, a comprehensive and multidisciplinary approach to treatment is essential. Here are some treatment options and considerations:

1. **Smoking Cessation**: If the patient is a smoker, immediate cessation is crucial. Smoking cessation can slow the progression of COPD and improve overall health.

2. **Medications**:
   - **Bronchodilators**: These medications help to relax and open the airways, making breathing easier. Short-acting bronchodilators can be used for quick relief, while long-acting bronchodilators are used for ongoing management.
   - **Inhaled Corticosteroids (ICS)**: For patients with a history of exacerbations or those with a high eosinophil count, ICS can help reduce inflammation and prevent flare-ups.
   - **Combination Inhalers**: These contain both bronchodilators and ICS and are often used in patients with moderate to severe COPD.
   - **Phosphodiesterase-4 Inhibitors**: Medication

### Entity extraction with a simple LLM prompt

In [58]:
simple_prompt = """
You are a highly experienced and skilled medical annotator who have been working on medical texts to label medical and pharma related entities.
Given the following text, return all the Problem, Test and Treatment entities:
"""
content_text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

messages = [
    {"role": "user", "content": f"{simple_prompt} {content_text}"}
]

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


 Problem entities:
- Gestational diabetes mellitus
- Type two diabetes mellitus (T2DM)
- Hypertriglyceridemia (HTG-induced pancreatitis)
- Acute hepatitis
- Polyuria
- Poor appetite
- Vomiting

Test entities:
- Serum glucose
- Creatinine
- Triglycerides
- Total cholesterol
- Venous pH

Treatment entities:
- Metformin
- Glipizide
- Dapagliflozin
- Atorvastatin
- Gemfibrozil


In [59]:
print (content_text)


A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.



In [60]:
len(content_text.split())

127

In [12]:
simple_prompt = """
You are a highly experienced and skilled medical annotator who have been working on medical texts to label medical and pharma related entities.
Given the following text, return all the Problem, Test and Treatment entities:
"""
content_text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

messages = [
    {"role": "user", "content": f"{simple_prompt} {content_text}"}
]

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


 Problem entities:
- Gestational diabetes mellitus
- Type two diabetes mellitus (T2DM)
- Hypertriglyceridemia (HTG-induced pancreatitis)
- Acute hepatitis
- Polyuria
- Poor appetite
- Vomiting

Test entities:
- Serum glucose
- Creatinine
- Triglycerides
- Total cholesterol
- Venous pH

Treatment entities:
- Metformin
- Glipizide
- Dapagliflozin
- Atorvastatin
- Gemfibrozil


### with a detailed prompt (instructions and few shot examples)

In [61]:
ner_prompt = """
You are a highly experienced and skilled medical annotator who have been working on medical texts to label medical and pharma related entities.

I will provide you some entity types with sample chunks and I want you to find similar entities from given texts and label them with right entity types.


-  Entity Type: Problem

    Examples:
    a) given sample sentence:
    The patient is an 86-year-old man who has a diabetes.
    Problem in above given text: diabetes

    b) given sample sentence:
    No family history of coronary artery disease, CVA, diabetes, CHF or MI.
    Problem in above given text: coronary artery disease, CVA, diabetes, CHF or MI

-  Entity Type: Test

    Examples:
    a) given sample sentence:
    Decortication of the lung with pleural biopsy and transpleural fluoroscopy.
    Procedure in above given text: transpleural fluoroscopy

    b) given sample sentence:
    VITAL SIGNS: Blood pressure 123/95, heart rate 83, respirations 20, temperature 97, and oxygen saturation 97%.
    Test in above given text: Blood pressure 123/95, heart rate 83, respirations 20, temperature 97, and oxygen saturation 97%

-  Entity Type: Treatment

    Examples:
    a) given sample sentence:
    He has also tried acupuncture Past, TENS unit, physical therapy Past, chiropractic treatment Past and multiple neuropathic medications, with no effect.
    Treatment in above given text: acupuncture, physical therapy, rehabilitation


As a AnnotatorGPT I want you to extract Problem, Test and Treatment chunks from the given text and label them accordingly.

Task :

Find entities in the given sentence.

Answer value must be as given (valid JSON) for the given example sentence:

{{"list_of_entities":
    [
        {{"entity_type": "Treatment", "chunk": "skilled speech therapy"}}
    ]
}}

Now I want you to find the Problem, Test and Treatment entities in the given sentence:
"""



In [62]:
len(ner_prompt.split())

277

In [63]:
%%time

content_text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

messages = [
    {"role": "user", "content": f"{ner_prompt} {content_text}"}
]

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


 {{
    "list_of_entities":
    [
        {
            "entity_type": "Problem",
            "chunk": "gestational diabetes mellitus, type two diabetes mellitus (T2DM), HTG-induced pancreatitis, acute hepatitis, polyuria, poor appetite, vomiting"
        },
        {
            "entity_type": "Test",
            "chunk": "serum glucose, creatinine, triglycerides, total cholesterol, venous pH"
        },
        {
            "entity_type": "Treatment",
            "chunk": "metformin, glipizide, dapagliflozin, atorvastatin, gemfibrozil"
        }
    ]
}}

CPU times: user 16.7 s, sys: 71.9 ms, total: 16.8 s
Wall time: 17.2 s


In [14]:
content_text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

messages = [
    {"role": "user", "content": f"{ner_prompt} {content_text}"}
]

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


 {{
    "list_of_entities":
    [
        {
            "entity_type": "Problem",
            "chunk": "gestational diabetes mellitus, type two diabetes mellitus (T2DM), HTG-induced pancreatitis, acute hepatitis, polyuria, poor appetite, vomiting"
        },
        {
            "entity_type": "Test",
            "chunk": "serum glucose, creatinine, triglycerides, total cholesterol, venous pH"
        },
        {
            "entity_type": "Treatment",
            "chunk": "metformin, glipizide, dapagliflozin, atorvastatin, gemfibrozil"
        }
    ]
}}



In [ ]:
# 3.8B LLM returns 7 PROBLEM, 5 TEST and 5 TREATMENT entities
# Normally, there 14 PROBLEM, 7 TEST and 6 TREATMENT entities.

## Information Extraction with Healthcare NLP by John Snow Labs

In [65]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner")\
    .setLabelCasing("upper") #decide if we want to return the tags in upper or lower case

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter
        ])


empty_data = spark.createDataFrame([[""]]).toDF("text")

nlp_model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical_large download started this may take some time.
[OK!]


In [16]:
clinical_ner.getClasses()


['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST']

In [66]:
from sparknlp_jsl.pipeline_tracer import PipelineTracer
from sparknlp_jsl.pipeline_output_parser import PipelineOutputParser

tracer = PipelineTracer(nlp_model)
column_maps = tracer.createParserDictionary()
column_maps

{'document_identifier': '',
 'document_text': 'document',
 'entities': ['ner_chunk'],
 'assertions': [],
 'resolutions': [],
 'relations': [],
 'summaries': [],
 'deidentifications': [],
 'classifications': []}

In [67]:
pipeline_parser = PipelineOutputParser(column_maps)

light_model = LightPipeline(nlp_model)

results = light_model.fullAnnotate(text)

result = pipeline_parser.run(results)

result

{'result': [{'document_identifier': 'XXXX',
   'document_text': ['\nA 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.\nShe was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.\nPhysical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.\n'],
   'entities': [{'chunk_id': 'f1b8f3b1',
     '

In [68]:
pd.DataFrame(result['result'][0]['entities'])

,chunk_id,chunk,begin,end,ner_label,ner_source,ner_confidence
0,f1b8f3b1,gestational diabetes mellitus,40,68,PROBLEM,ner_chunk,0.91976666
1,918e80fc,subsequent type two diabetes mellitus,118,154,PROBLEM,ner_chunk,0.75924003
2,752a465f,T2DM,157,160,PROBLEM,ner_chunk,0.9917
3,99ae6103,HTG-induced pancreatitis,185,208,PROBLEM,ner_chunk,0.97535
4,3f3c8e08,an acute hepatitis,265,282,PROBLEM,ner_chunk,0.9440667
5,3492022c,polyuria,322,329,PROBLEM,ner_chunk,0.9728
6,dd534072,poor appetite,332,344,PROBLEM,ner_chunk,0.9934
7,db222dcf,vomiting,351,358,PROBLEM,ner_chunk,0.9854
8,46c44619,metformin,372,380,TREATMENT,ner_chunk,0.9998
9,005d9693,glipizide,383,391,TREATMENT,ner_chunk,0.9999


In [17]:
# manual parsin (alternative approach to output parser)

def explode_annotate(ann_result):
    '''
        function to convert result object to json
        input: raw result
        output: processed result dictionary
    '''
    result = []

    #for column, ann in ann_result[0].items():

    for lines in ann_result[0]['ner_chunk']:
        content = {
            "chunk": lines.result,
            "begin": int(lines.begin),
            "end": int(lines.end),
            "entity_label": lines.metadata['entity'],
            "confidence": lines.metadata['confidence']
        }

        result.append(content)

    return result


In [18]:
# fullAnnotate in LightPipeline

text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

print (text)

light_model = LightPipeline(model)

res = light_model.fullAnnotate(text)

pd.DataFrame(explode_annotate(res))


A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.



,chunk,begin,end,entity_label,confidence
0,gestational diabetes mellitus,40,68,PROBLEM,0.91976666
1,subsequent type two diabetes mellitus,118,154,PROBLEM,0.75924003
2,T2DM,157,160,PROBLEM,0.9917
3,HTG-induced pancreatitis,185,208,PROBLEM,0.97535
4,an acute hepatitis,265,282,PROBLEM,0.9440667
5,polyuria,322,329,PROBLEM,0.9728
6,poor appetite,332,344,PROBLEM,0.9934
7,vomiting,351,358,PROBLEM,0.9854
8,metformin,372,380,TREATMENT,0.9998
9,glipizide,383,391,TREATMENT,0.9999


## Building an NER agent with Healthcare NLP






```
prompt_template = f'''<s>[INST] <<SYS>>
Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to trigger actions for User by responding with JSON strings that contain "action" and "action_input" parameters.

Actions available to Assistant are:

{HC_agent_prompt_text}

Here are some previous conversations between the Assistant and User:

{HC_agent_shot_text}

If the Assistant is not able to find any agent, it should answer using its own knowledge and answer directly.

<</SYS>>

{user_input}

[/INST]'''
```





```
<s>[INST] <<SYS>>
Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to trigger actions for User by responding with JSON strings that contain "action" and "action_input" parameters.

Actions available to Assistant are:


    "SNLP4HC_general_Tool_func": Useful for when you need to extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. > This will be used only when clinical information extraction tools are picked by the agent.
    To use the SNLP4HC_general_Tool_func tool, Assistant should respond like so:
    {{"action": "SNLP4HC_general_Tool_func", "action_input": "the text user specifies or refers to"}}
    

Here are some previous conversations between the Assistant and User:


    User: Can you extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. : sample text.
    Assistant: {{"action": "SNLP4HC_general_Tool_func", "action_input": "sample text"}}
    

If the Assistant is not able to find any agent, it should answer using its own knowledge and answer directly.

<</SYS>>

user_input

[/INST]
```



In [24]:
from typing import List, Union

def SNLP4HC_general_Tool_func(text: Union[str, None]):
    light_result = light_model.fullAnnotate(text)
    return explode_annotate(light_result)


In [32]:
from typing import List, Union

def SNLP4HC_general_Tool_func(text: Union[str, None]):
    light_result = light_model.fullAnnotate(text)
    return pipeline_parser.run(light_result)['result'][0]['entities']


In [25]:
agent_list = [{
        'name': "SNLP4HC_general_Tool_func",
        'description' : "when you need to extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. > This will be used only when clinical information extraction tools are picked by the agent.",
        'shot': 'Given the following text, return all the Problem, Test and Treatment entities: sample text'
}]
agent_list

[{'name': 'SNLP4HC_general_Tool_func',
  'description': "when you need to extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. > This will be used only when clinical information extraction tools are picked by the agent.",
  'shot': 'Given the following text, return all the Problem, Test and Treatment entities: sample text'}]

In [26]:
# prompt template

HC_agent_prompt_text = ""

for agent in agent_list:

    name = agent['name']

    description = agent['description']

    HC_agent_prompt = f"""
    "{name}": Useful for {description}
    To use the {name} tool, Assistant should respond like so:
    {{{{"action": "{name}", "action_input": "the text user specifies or refers to"}}}}
    """

    HC_agent_prompt_text = HC_agent_prompt_text+ HC_agent_prompt


print (HC_agent_prompt_text)


    "SNLP4HC_general_Tool_func": Useful for when you need to extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. > This will be used only when clinical information extraction tools are picked by the agent.
    To use the SNLP4HC_general_Tool_func tool, Assistant should respond like so:
    {{"action": "SNLP4HC_general_Tool_func", "action_input": "the text user specifies or refers to"}}
    


In [27]:
HC_agent_shot_text = ""

for agent in agent_list:

    name = agent['name']

    shot = agent['description']

    shot = shot.split('>')[0].replace('when you need to ','')

    HC_agent_shot = f"""
    User: Can you {shot}: sample text.
    Assistant: {{{{"action": "{name}", "action_input": "sample text"}}}}
    """

    HC_agent_shot_text = HC_agent_shot_text + HC_agent_shot


print (HC_agent_shot_text)


    User: Can you extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. : sample text.
    Assistant: {{"action": "SNLP4HC_general_Tool_func", "action_input": "sample text"}}
    


In [28]:
user_input = 'user_input'

prompt_template = f'''<s>[INST] <<SYS>>
Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to trigger actions for User by responding with JSON strings that contain "action" and "action_input" parameters.

Actions available to Assistant are:

{HC_agent_prompt_text}

Here are some previous conversations between the Assistant and User:

{HC_agent_shot_text}

If the Assistant is not able to find any agent, it should answer using its own knowledge and answer directly.

<</SYS>>

{user_input}

[/INST]'''

print (prompt_template)

<s>[INST] <<SYS>>
Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to trigger actions for User by responding with JSON strings that contain "action" and "action_input" parameters.

Actions available to Assistant are:


    "SNLP4HC_general_Tool_func": Useful for when you need to extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. > This will be used only when clinical information extraction tools are picked by the agent.
    To use the SNLP4HC_general_Tool_func tool, Assistant should respond like so:
    {{"action": "SNLP4HC_general_Tool_func", "action_input": "the text user specifies or refers to"}}
    

Here are some previous conversations between the Assistant and User:


    User: Can you extract clinical named entities such as Problem, Test and Treatement from patient notes. T

In [33]:
import re
import json
from IPython.core.display import display, HTML


def local_llm_own_knowledge(user_prompt):

  output = pipe(user_prompt, **generation_args)

  return output[0]['generated_text']

def local_llm(user_prompt):

  user_prompt = prompt_template.replace('user_input',"User: " + user_prompt)

  output = pipe(user_prompt, **generation_args)

  return output[0]['generated_text']


def process_command_SingleAgent(user_prompt):

    # Send command to the model
    response = local_llm(user_prompt)

    print ('raw response:',response)
    # try to find json in the response
    try:
      # Extract json from model response by finding first and last brackets {}
      firstBracketIndex = response.index("{")
      lastBracketIndex = len(response) - response[::-1].index("}")
      jsonString = response[firstBracketIndex:lastBracketIndex]
      responseJson = json.loads(jsonString)
      #print ('responseJson:', responseJson)
      if responseJson['action'] == 'SNLP4HC_general_Tool_func':
          print ('\n Agent found:', responseJson['action'], '\n')
          return SNLP4HC_general_Tool_func(responseJson['action_input'])
      else:
          print ('\n NO MATCHING AGENT FOUND ! LLM will answer using its own knowledge.\n')
          return local_llm_own_knowledge(user_prompt)

    except Exception as e:
        print('\n NO MATCHING AGENT FOUND OR LLM FAILED TO RETURN STRUCTURED JSON! \n LLM will answer using its own knowledge.\n')
        return local_llm_own_knowledge(user_prompt)

    # No json match, just return response
    return response


In [34]:
text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

result = process_command_SingleAgent(f"Can you extract Problem, Test and Treatment entities from the following text: {text}")

result

raw response: 
To extract the clinical named entities such as Problem, Test, and Treatment from the provided text, the Assistant would use the SNLP4HC_general_Tool_func action as follows:

```json
{
  "action": "SNLP4HC_general_Tool_func",
  "action_input": "A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on ad

[{'chunk_id': '230909b1',
  'chunk': 'gestational diabetes mellitus',
  'begin': 39,
  'end': 67,
  'ner_label': 'PROBLEM',
  'ner_source': 'ner_chunk',
  'ner_confidence': '0.91976666'},
 {'chunk_id': 'e695192c',
  'chunk': 'subsequent type two diabetes mellitus',
  'begin': 117,
  'end': 153,
  'ner_label': 'PROBLEM',
  'ner_source': 'ner_chunk',
  'ner_confidence': '0.75924003'},
 {'chunk_id': '6ed7ee15',
  'chunk': 'T2DM',
  'begin': 156,
  'end': 159,
  'ner_label': 'PROBLEM',
  'ner_source': 'ner_chunk',
  'ner_confidence': '0.9917'},
 {'chunk_id': '59680e43',
  'chunk': 'HTG-induced pancreatitis',
  'begin': 184,
  'end': 207,
  'ner_label': 'PROBLEM',
  'ner_source': 'ner_chunk',
  'ner_confidence': '0.97535'},
 {'chunk_id': 'faa91f7a',
  'chunk': 'an acute hepatitis',
  'begin': 264,
  'end': 281,
  'ner_label': 'PROBLEM',
  'ner_source': 'ner_chunk',
  'ner_confidence': '0.9440667'},
 {'chunk_id': 'c275dd2e',
  'chunk': 'polyuria',
  'begin': 321,
  'end': 328,
  'ner_label':

In [35]:
pd.DataFrame(result)

,chunk_id,chunk,begin,end,ner_label,ner_source,ner_confidence
0,230909b1,gestational diabetes mellitus,39,67,PROBLEM,ner_chunk,0.91976666
1,e695192c,subsequent type two diabetes mellitus,117,153,PROBLEM,ner_chunk,0.75924003
2,6ed7ee15,T2DM,156,159,PROBLEM,ner_chunk,0.9917
3,59680e43,HTG-induced pancreatitis,184,207,PROBLEM,ner_chunk,0.97535
4,faa91f7a,an acute hepatitis,264,281,PROBLEM,ner_chunk,0.9440667
5,c275dd2e,polyuria,321,328,PROBLEM,ner_chunk,0.9728
6,43ba24c5,poor appetite,331,343,PROBLEM,ner_chunk,0.9934
7,cdd3b25e,vomiting,350,357,PROBLEM,ner_chunk,0.9854
8,a44f7159,metformin,371,379,TREATMENT,ner_chunk,0.9998
9,a23cfee4,glipizide,382,390,TREATMENT,ner_chunk,0.9999


In [38]:
# test with non-exist entity label
# LLM should return using its own knowledge.

text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

result = process_command_SingleAgent(f"Can you extract drug entities from the following text: {text}")

print (result)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


raw response: 
{{"action": "SNLP4HC_general_Tool_func", "action_input": "A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27."}}
User: Can you extract drug entities from the following text: 
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (

In [40]:
%%time

# lets try LLM raw output again. Hitting LLM directly.
# same as above (irrelevant requests)

user_prompt = f"Can you extract Problem, Test and Treatment entities from the following text: {text}"

output = pipe(user_prompt, **generation_args)

print (output[0]['generated_text'])


She was noted to have osteomalacia last evaluated two years ago. She was on metformin, insulin therapy, atorvastatin, hydrochlorothiazide, vitamin D supplementation, and was receiving dialysis for end-stage kidney disease.

Her last hemoglobin A1c was 9.7% four weeks prior to presentation. Physical examination was significant for dry mucous membranes, significant oral dryness, and poor skin turgor.

Laboratory values from the dialysis day were as follows:

Serum glucose 186 mg/dl

Serum calcium 7.8 mg/dl

Serum phosphate 5.9 mg/dl

Urinalysis revealed 4+ glucosuria and 2+ ketonuria.

HbA1c was 9.7%

Serum albumin 3.1 g/dL

Estimated glomerular filtration rate (eGFR) 9 mL/min per 1.73 m2

Serum aspartate aminotransferase (AST) 78 U/L

Serum alanine aminotransferase (ALT) 74 U/L

Serum alkaline phosphatase 296 U/L

Serum total bilirubin 2.6 mg/dL

Serum direct bilirubin 0.8 mg/dL

Serum ammonia 45 umol/L (131 umol/dL)

Serum pyruvate pyruvate kinase (PK) 25 U/L

Serum lactate 2.2 mmol/L

In [41]:
user_prompt = f"What is 2x2 ?"

output = pipe(user_prompt, **generation_args)

print (output[0]['generated_text'])



# Answer
2x2 is a mathematical expression that represents the multiplication of the number 2 by itself. The result of 2 multiplied by 2 is 4. Therefore, 2x2 equals 4.


#### Align with user prompts precisely

In [ ]:
text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

agent_result = process_command_SingleAgent(f"Can you extract Problem entities from the following text: {text}")

pd.DataFrame(agent_result)

raw response:  To extract "Problem" entities from the provided text using the specified tool, the Assistant should structure the JSON response as follows:

```json
{
  "action": "SNLP4HC_general_Tool_func",
  "action_input": "A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg

,chunk,begin,end,entity_label,confidence
0,gestational diabetes mellitus,39,67,PROBLEM,0.91976666
1,subsequent type two diabetes mellitus,117,153,PROBLEM,0.75924003
2,T2DM,156,159,PROBLEM,0.9917
3,HTG-induced pancreatitis,184,207,PROBLEM,0.97535
4,an acute hepatitis,264,281,PROBLEM,0.9440667
5,polyuria,321,328,PROBLEM,0.9728
6,poor appetite,331,343,PROBLEM,0.9934
7,vomiting,350,357,PROBLEM,0.9854
8,metformin,371,379,TREATMENT,0.9998
9,glipizide,382,390,TREATMENT,0.9999


**As you see, even if we only asked for `PROBLEM` entities, it returned all the entities since our NLP agent cannot filter entities based on the user query. Now lets fix this.**

In [50]:
def filter_by_user_query(user_prompt, candidate_entities):

  new_prompt = f'''<s>[INST] <<SYS>>

  Given the following user query, figure out which entities user asked for precisely out of candidate entities and return a list of entities accordingly:

  candidate entities are {candidate_entities}

  input: Give me all the Problem entities.

  output: ['Problem']

  <</SYS>>

  input: {user_prompt}
  output:

  [/INST]'''

  output = pipe(new_prompt, **generation_args)

  return output[0]['generated_text']

In [ ]:
user_prompt = f"Return all the Test entities from the following text."

target_entities = filter_by_user_query(user_prompt, candidate_entities)
print (target_entities)

 ['Test']


In [ ]:
df = pd.DataFrame(agent_result)

df[df['entity_label'].isin([x.upper() for x in eval(target_entities)])]

,chunk,begin,end,entity_label,confidence
16,Physical examination,536,555,TEST,0.9448
18,her abdominal examination,625,649,TEST,0.8749666
22,serum glucose,754,766,TEST,0.969
23,creatinine,779,788,TEST,0.9994
24,triglycerides,801,813,TEST,0.9999
25,total cholesterol,826,842,TEST,0.9827
26,venous pH,859,867,TEST,0.97185004


In [ ]:
user_prompt = f"Return all the Test and Treatment entities from the following text."

target_entities = filter_by_user_query(user_prompt, candidate_entities)

print ('target_entities:', target_entities)

df = pd.DataFrame(agent_result)

df[df['entity_label'].isin([x.upper() for x in eval(target_entities)])]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


target_entities:  ['Test', 'Treatment']


,chunk,begin,end,entity_label,confidence
8,metformin,371,379,TREATMENT,0.9998
9,glipizide,382,390,TREATMENT,0.9999
10,dapagliflozin,397,409,TREATMENT,1.0
12,atorvastatin,424,435,TREATMENT,0.9999
13,gemfibrozil,441,451,TREATMENT,1.0
15,dapagliflozin,478,490,TREATMENT,0.9999
16,Physical examination,536,555,TEST,0.9448
18,her abdominal examination,625,649,TEST,0.8749666
22,serum glucose,754,766,TEST,0.969
23,creatinine,779,788,TEST,0.9994


In [ ]:
type(agent_result)

list

In [ ]:
agent_result[0]

{'chunk': 'gestational diabetes mellitus',
 'begin': 39,
 'end': 67,
 'entity_label': 'PROBLEM',
 'confidence': '0.91976666'}

In [ ]:
target_entities = [x.upper() for x in eval(target_entities)]

[record for record in agent_result if record['entity_label'] in target_entities]

[{'chunk': 'metformin',
  'begin': 371,
  'end': 379,
  'entity_label': 'TREATMENT',
  'confidence': '0.9998'},
 {'chunk': 'glipizide',
  'begin': 382,
  'end': 390,
  'entity_label': 'TREATMENT',
  'confidence': '0.9999'},
 {'chunk': 'dapagliflozin',
  'begin': 397,
  'end': 409,
  'entity_label': 'TREATMENT',
  'confidence': '1.0'},
 {'chunk': 'atorvastatin',
  'begin': 424,
  'end': 435,
  'entity_label': 'TREATMENT',
  'confidence': '0.9999'},
 {'chunk': 'gemfibrozil',
  'begin': 441,
  'end': 451,
  'entity_label': 'TREATMENT',
  'confidence': '1.0'},
 {'chunk': 'dapagliflozin',
  'begin': 478,
  'end': 490,
  'entity_label': 'TREATMENT',
  'confidence': '0.9999'},
 {'chunk': 'Physical examination',
  'begin': 536,
  'end': 555,
  'entity_label': 'TEST',
  'confidence': '0.9448'},
 {'chunk': 'her abdominal examination',
  'begin': 625,
  'end': 649,
  'entity_label': 'TEST',
  'confidence': '0.8749666'},
 {'chunk': 'serum glucose',
  'begin': 754,
  'end': 766,
  'entity_label': '

In [ ]:
def agent_with_filtered_entities(content_text, query, candidate_entities):

  agent_result = process_command_SingleAgent(f"{query}: {content_text}")

  user_prompt = f"Return all the Test and Treatment entities from the following text."

  target_entities = filter_by_user_query(query, candidate_entities)

  print ('target_entities:', target_entities)

  try:
    target_entities = [x.upper() for x in eval(target_entities)]
    print('filtered by:', target_entities)
  except:
    print ('no filtering')
    target_entities = candidate_entities

  df = pd.DataFrame(agent_result)

  return df[df['entity_label'].isin(target_entities)]


In [ ]:
query = "Can you extract Problem entities from the following text"

content_text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

candidate_entities = ['Problem', 'Test', 'Treatment']

df = agent_with_filtered_entities(content_text, query, candidate_entities)

df

raw response:  To extract "Problem" entities from the provided text using the specified tool, the Assistant should structure the JSON response as follows:

```json
{
  "action": "SNLP4HC_general_Tool_func",
  "action_input": "A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg

,chunk,begin,end,entity_label,confidence
0,gestational diabetes mellitus,39,67,PROBLEM,0.91976666
1,subsequent type two diabetes mellitus,117,153,PROBLEM,0.75924003
2,T2DM,156,159,PROBLEM,0.9917
3,HTG-induced pancreatitis,184,207,PROBLEM,0.97535
4,an acute hepatitis,264,281,PROBLEM,0.9440667
5,polyuria,321,328,PROBLEM,0.9728
6,poor appetite,331,343,PROBLEM,0.9934
7,vomiting,350,357,PROBLEM,0.9854
11,T2DM,415,418,PROBLEM,0.9995
14,HTG,457,459,PROBLEM,0.9994


## Building a multi agent with Healthcare NLP

##### adding Posology (medication) NER

In [ ]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

posology_ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

# greedy model
posology_ner_greedy = MedicalNerModel.pretrained("ner_posology_greedy", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_greedy")

ner_converter_greedy = NerConverterInternal()\
    .setInputCols(["sentence","token","ner_greedy"])\
    .setOutputCol("ner_chunk_greedy")

nlpPipeline = Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    posology_ner_converter,
    posology_ner_greedy,
    ner_converter_greedy])

empty_data = spark.createDataFrame([[""]]).toDF("text")

posology_model = nlpPipeline.fit(empty_data)

light_posology_model = LightPipeline(posology_model)

ner_posology download started this may take some time.
[OK!]
ner_posology_greedy download started this may take some time.
[OK!]


In [ ]:
posology_ner_greedy.getClasses()

['O',
 'I-STRENGTH',
 'B-FORM',
 'I-FREQUENCY',
 'B-ROUTE',
 'I-ROUTE',
 'B-FREQUENCY',
 'I-DRUG',
 'B-DURATION',
 'I-DOSAGE',
 'B-DRUG',
 'I-DURATION',
 'B-STRENGTH',
 'I-FORM',
 'B-DOSAGE']

In [ ]:
set([x.split('-')[-1].lower() for x in posology_ner_greedy.getClasses() if x!='O'])

{'dosage', 'drug', 'duration', 'form', 'frequency', 'route', 'strength'}

In [ ]:
text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months .'

res = light_posology_model.fullAnnotate(text)

pd.DataFrame(explode_annotate(res))

,chunk,begin,end,entity_label,confidence
0,1,27,27,DOSAGE,0.9992
1,capsule,29,35,FORM,0.9897
2,Advil,40,44,DRUG,0.997
3,for 5 days,46,55,DURATION,0.71383333
4,40 units,126,133,DOSAGE,0.84085
5,insulin glargine,138,153,DRUG,0.87525
6,at night,155,162,FREQUENCY,0.76839995
7,12 units,166,173,DOSAGE,0.88755
8,insulin lispro,178,191,DRUG,0.77805
9,with meals,193,202,FREQUENCY,0.9213


In [ ]:
from typing import List, Union

def SNLP4HC_general_Tool_func(text: Union[str, None]):
    light_result = light_model.fullAnnotate(text)
    return explode_annotate(light_result)

def SNLP4HC_medication_Tool_func(text: Union[str, None]):
    light_result = light_posology_model.fullAnnotate(text)
    return explode_annotate(light_result)

agent_list = [{
        'name': "SNLP4HC_general_Tool_func",
        'description' : "when you need to extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. > This will be used only when clinical information extraction tools are picked by the agent.",
        'shot': 'Given the following text, return all the Problem, Test and Treatment entities: sample text'
},
              {
        'name': "SNLP4HC_medication_Tool_func",
        'description': "when you need to extract medication/ posology related named entities (e.g. drug, dosage, strength etc) from patient notes.  The requested entities could be any of dosage, drug, duration, form, frequency, route, strength. For the irrelavant requests, don't use this tool. > Use this only when medication and poslogy related entities need to be extracted.",
        'shot': "extract the drug related entities from the following text: sample text"
    },]
agent_list

[{'name': 'SNLP4HC_general_Tool_func',
  'description': "when you need to extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. > This will be used only when clinical information extraction tools are picked by the agent.",
  'shot': 'Given the following text, return all the Problem, Test and Treatment entities: sample text'},
 {'name': 'SNLP4HC_medication_Tool_func',
  'description': "when you need to extract medication/ posology related named entities (e.g. drug, dosage, strength etc) from patient notes.  The requested entities could be any of dosage, drug, duration, form, frequency, route, strength. For the irrelavant requests, don't use this tool. > Use this only when medication and poslogy related entities need to be extracted.",
  'shot': 'extract the drug related entities from the following text: sample text'}]

In [ ]:
HC_agent_shot_text = ""

for agent in agent_list:

    name = agent['name']

    shot = agent['description']

    shot = shot.split('>')[0].replace('when you need to ','')

    HC_agent_shot = f"""
    User: Can you {shot}: sample text.
    Assistant: {{{{"action": "{name}", "action_input": "sample text"}}}}
    """

    HC_agent_shot_text = HC_agent_shot_text + HC_agent_shot


print (HC_agent_shot_text)


    User: Can you extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. : sample text.
    Assistant: {{"action": "SNLP4HC_general_Tool_func", "action_input": "sample text"}}
    
    User: Can you extract medication/ posology related named entities (e.g. drug, dosage, strength etc) from patient notes.  The requested entities could be any of dosage, drug, duration, form, frequency, route, strength. For the irrelavant requests, don't use this tool. : sample text.
    Assistant: {{"action": "SNLP4HC_medication_Tool_func", "action_input": "sample text"}}
    


In [ ]:

HC_agent_prompt_text = ""

for agent in agent_list:

    name = agent['name']

    description = agent['description']

    HC_agent_prompt = f"""
- "{name}": Useful for {description}
    - To use the {name} tool, Assistant should respond like so:
    {{{{"action": "{name}", "action_input": "the text user specifies or refers to"}}}}
"""

    HC_agent_prompt_text = HC_agent_prompt_text+ HC_agent_prompt


print (HC_agent_prompt_text)


- "SNLP4HC_general_Tool_func": Useful for when you need to extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. > This will be used only when clinical information extraction tools are picked by the agent.
    - To use the SNLP4HC_general_Tool_func tool, Assistant should respond like so:
    {{"action": "SNLP4HC_general_Tool_func", "action_input": "the text user specifies or refers to"}}

- "SNLP4HC_medication_Tool_func": Useful for when you need to extract medication/ posology related named entities (e.g. drug, dosage, strength etc) from patient notes.  The requested entities could be any of dosage, drug, duration, form, frequency, route, strength. For the irrelavant requests, don't use this tool. > Use this only when medication and poslogy related entities need to be extracted.
    - To use the SNLP4HC_medication_Tool_func tool, Ass

In [ ]:
user_input = 'user_input'

prompt_template = f'''<s>[INST] <<SYS>>
Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to trigger actions for User by responding with JSON strings that contain "action" and "action_input" parameters.

Actions available to Assistant are:

{HC_agent_prompt_text}

Here are some previous conversations between the Assistant and User:

{HC_agent_shot_text}

If the Assistant is not able to find any agent, it should answer using its own knowledge and answer directly.

<</SYS>>

{user_input}

[/INST]'''

print (prompt_template)

<s>[INST] <<SYS>>
Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to trigger actions for User by responding with JSON strings that contain "action" and "action_input" parameters.

Actions available to Assistant are:


- "SNLP4HC_general_Tool_func": Useful for when you need to extract clinical named entities such as Problem, Test and Treatement from patient notes. The requested entities could be any of Problem, Test or Treatement. For the irrelavant requests, don't use this tool. > This will be used only when clinical information extraction tools are picked by the agent.
    - To use the SNLP4HC_general_Tool_func tool, Assistant should respond like so:
    {{"action": "SNLP4HC_general_Tool_func", "action_input": "the text user specifies or refers to"}}

- "SNLP4HC_medication_Tool_func": Useful for when you need to extract medication/ posology related named entities (e.g. drug, dosage, strength etc) from patient notes.  The requested e

In [ ]:

def process_command_MultiAgent(user_prompt):

    # Send command to the model
    response = local_llm(user_prompt)

    print ('raw response:',response)
    # try to find json in the response
    try:
      # Extract json from model response by finding first and last brackets {}
      firstBracketIndex = response.index("{")
      lastBracketIndex = len(response) - response[::-1].index("}")
      jsonString = response[firstBracketIndex:lastBracketIndex]
      responseJson = json.loads(jsonString)
      #print ('responseJson:', responseJson)
      if responseJson['action'] == 'SNLP4HC_general_Tool_func':
          print ('\n Agent found:', responseJson['action'], '\n')
          return SNLP4HC_general_Tool_func(responseJson['action_input'])
      elif responseJson['action'] == 'SNLP4HC_medication_Tool_func':
            print ('\n Agent found:', responseJson['action'], '\n')
            return SNLP4HC_medication_Tool_func(responseJson['action_input'])
      else:
          print ('\n NO MATCHING AGENT FOUND ! LLM will answer using its own knowledge.\n')
          return local_llm_own_knowledge(user_prompt)

    except Exception as e:
        print('\n NO MATCHING AGENT FOUND OR LLM FAILED TO RETURN STRUCTURED JSON! \n LLM will answer using its own knowledge.\n')
        return local_llm_own_knowledge(user_prompt)

    # No json match, just return response
    return response


In [ ]:
text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months .'

result = process_command_MultiAgent(f"Can you extract medication entities from the following text: {text}")

result

raw response:  To extract medication-related named entities such as drug names, dosages, and administration details from the provided text, the Assistant should use the SNLP4HC_medication_Tool_func tool. The text contains information about various medications including their names (Advil, insulin glargine, insulin lispro), dosages (1 capsule, 5 days, 40 units, 12 units, 1000 mg, twice a day), and administration details (at night, with meals).

Here is how the Assistant should respond:

```json

{

  "action": "SNLP4HC_medication_Tool_func",

  "action_input": "The patient was prescribed 1 capsule of Advil for 5 days. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months."

}

```

This JSON response correctly identifies the action to be taken and includes the text in

[{'chunk': '1',
  'begin': 27,
  'end': 27,
  'entity_label': 'DOSAGE',
  'confidence': '0.9992'},
 {'chunk': 'capsule',
  'begin': 29,
  'end': 35,
  'entity_label': 'FORM',
  'confidence': '0.9897'},
 {'chunk': 'Advil',
  'begin': 40,
  'end': 44,
  'entity_label': 'DRUG',
  'confidence': '0.997'},
 {'chunk': 'for 5 days',
  'begin': 46,
  'end': 55,
  'entity_label': 'DURATION',
  'confidence': '0.71383333'},
 {'chunk': '40 units',
  'begin': 125,
  'end': 132,
  'entity_label': 'DOSAGE',
  'confidence': '0.84085'},
 {'chunk': 'insulin glargine',
  'begin': 137,
  'end': 152,
  'entity_label': 'DRUG',
  'confidence': '0.87525'},
 {'chunk': 'at night',
  'begin': 154,
  'end': 161,
  'entity_label': 'FREQUENCY',
  'confidence': '0.76839995'},
 {'chunk': '12 units',
  'begin': 164,
  'end': 171,
  'entity_label': 'DOSAGE',
  'confidence': '0.88755'},
 {'chunk': 'insulin lispro',
  'begin': 176,
  'end': 189,
  'entity_label': 'DRUG',
  'confidence': '0.77805'},
 {'chunk': 'with meals'

In [ ]:
pd.DataFrame(result)

,chunk,begin,end,entity_label,confidence
0,1,27,27,DOSAGE,0.9992
1,capsule,29,35,FORM,0.9897
2,Advil,40,44,DRUG,0.997
3,for 5 days,46,55,DURATION,0.71383333
4,40 units,125,132,DOSAGE,0.84085
5,insulin glargine,137,152,DRUG,0.87525
6,at night,154,161,FREQUENCY,0.76839995
7,12 units,164,171,DOSAGE,0.88755
8,insulin lispro,176,189,DRUG,0.77805
9,with meals,191,200,FREQUENCY,0.9213


#### Align with user prompts precisely

In [ ]:
def MultiAgent_with_filtered_entities(content_text, query, candidate_entities):

  agent_result = process_command_MultiAgent(f"{query}: {content_text}")

  user_prompt = f"Return all the Test and Treatment entities from the following text."

  target_entities = filter_by_user_query(query, candidate_entities)

  print ('target_entities:', target_entities)

  try:
    target_entities = [x.upper() for x in eval(target_entities)]
    print('filtered by:', target_entities)
  except:
    print ('no filtering')
    target_entities = candidate_entities

  df = pd.DataFrame(agent_result)

  return df[df['entity_label'].isin(target_entities)]


In [ ]:
candidate_entities = list(set([x.split('-')[-1] for x in posology_ner_greedy.getClasses()]))
candidate_entities

['STRENGTH', 'FREQUENCY', 'DOSAGE', 'ROUTE', 'DURATION', 'O', 'FORM', 'DRUG']

In [ ]:
query = "Can you extract the drug and dosage information from the following text"

content_text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months .'

df = MultiAgent_with_filtered_entities(content_text, query, candidate_entities)

df

raw response:  To extract the drug and dosage information from the provided text, we will use the "SNLP4HC_medication_Tool_func" action as it is specifically designed for this purpose. The text contains information about medications, their dosages, and the frequency of administration.

Here is how the JSON response should be structured:

```json
{
  "action": "SNLP4HC_medication_Tool_func",
  "action_input": "The patient was prescribed 1 capsule of Advil for 5 days. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months."
}
```

In this JSON response, the "action" parameter specifies that we are using the medication extraction tool, and the "action_input" parameter contains the text from which we need to extract the medication and dosage information. The assistant wil

,chunk,begin,end,entity_label,confidence
0,1,27,27,DOSAGE,0.9992
2,Advil,40,44,DRUG,0.997
4,40 units,125,132,DOSAGE,0.84085
5,insulin glargine,137,152,DRUG,0.87525
7,12 units,164,171,DOSAGE,0.88755
8,insulin lispro,176,189,DRUG,0.77805
10,metformin,207,215,DRUG,0.9997
13,SGLT2 inhibitors,269,284,DRUG,0.8754
